# Atelier LLM #3 — NER en français avec Hugging Face

Notebook accompagnant l'article Medium et le dépôt *llm-learning-journey*. Il illustre l'usage de CamemBERT fine-tuné pour la reconnaissance d'entités nommées (NER).

In [1]:
# !pip install -U transformers datasets torch
from transformers import pipeline
from typing import List, Dict
import pandas as pd

In [2]:
ner = pipeline(
    task="ner",
    model="Jean-Baptiste/camembert-ner",
    aggregation_strategy="simple",
)

sample_text = "Le président Emmanuel Macron s'est rendu à Marseille pour discuter avec Orange et SFR."
ner(sample_text)[:3]

config.json:   0%|          | 0.00/892 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/269 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Device set to use cpu


[{'entity_group': 'PER',
  'score': np.float32(0.9985533),
  'word': 'Emmanuel Macron',
  'start': 12,
  'end': 28},
 {'entity_group': 'LOC',
  'score': np.float32(0.99789125),
  'word': 'Marseille',
  'start': 42,
  'end': 52},
 {'entity_group': 'ORG',
  'score': np.float32(0.97817916),
  'word': 'Orange',
  'start': 71,
  'end': 78}]

In [3]:
def pretty_entities(text: str, ents: List[Dict]):
    print(f"\nTexte: {text}")
    for e in ents:
        print(f" - {e['word']}\t({e['entity_group']})\tscore={e['score']:.3f}")

pretty_entities(sample_text, ner(sample_text))


Texte: Le président Emmanuel Macron s'est rendu à Marseille pour discuter avec Orange et SFR.
 - Emmanuel Macron	(PER)	score=0.999
 - Marseille	(LOC)	score=0.998
 - Orange	(ORG)	score=0.978
 - SFR	(ORG)	score=0.977


In [4]:
comments = [
    "J'adore le service de Free à Lyon !",
    "Le réseau SFR est catastrophique à Marseille...",
    "Orange propose enfin la 5G à Paris.",
    "Bouygues Telecom améliore ses offres."
]

for c in comments:
    ents = ner(c)
    pretty_entities(c, ents)


Texte: J'adore le service de Free à Lyon !
 - Free	(ORG)	score=0.986
 - Lyon	(LOC)	score=0.992

Texte: Le réseau SFR est catastrophique à Marseille...
 - SFR	(ORG)	score=0.985
 - Marseille	(LOC)	score=0.995

Texte: Orange propose enfin la 5G à Paris.
 - Orange	(ORG)	score=0.991
 - 5G	(MISC)	score=0.797
 - Paris	(LOC)	score=0.995

Texte: Bouygues Telecom améliore ses offres.
 - Bouygues Telecom	(ORG)	score=0.994


In [6]:
from transformers import pipeline as hf_pipeline

# NER (si tu l'utilises plus bas)
ner = hf_pipeline(
    "ner",
    model="Jean-Baptiste/camembert-ner",
    aggregation_strategy="simple"
)

# Sentiment (équivalent PT du tf-allocine)
sentiment = hf_pipeline(
    "sentiment-analysis",
    model="philschmid/pt-tblard-tf-allocine"  # <- fork PyTorch
    # optionnel : device=0 si tu as un GPU CUDA
)

def entity_sentiment_rows(text: str):
    ents = ner(text)
    s = sentiment(text)[0]
    return [
        {
            "text": text,
            "entity": e["word"],
            "label": e["entity_group"],
            "score": float(e["score"]),
            "sentiment_label": s["label"],
            "sentiment_score": float(s["score"]),
        }
        for e in ents
    ]

rows = []
for c in comments:
    rows.extend(entity_sentiment_rows(c))

df = pd.DataFrame(rows)
df.head()

Device set to use cpu


config.json:   0%|          | 0.00/896 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/811k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

Device set to use cpu


,text,entity,label,score,sentiment_label,sentiment_score
0,J'adore le service de Free à Lyon !,Free,ORG,0.985591,POSITIVE,0.960610
1,J'adore le service de Free à Lyon !,Lyon,LOC,0.991787,POSITIVE,0.960610
2,Le réseau SFR est catastrophique à Marseille...,SFR,ORG,0.984771,NEGATIVE,0.955451
3,Le réseau SFR est catastrophique à Marseille...,Marseille,LOC,0.995413,NEGATIVE,0.955451
4,Orange propose enfin la 5G à Paris.,Orange,ORG,0.990817,POSITIVE,0.648676


In [8]:
output_path = "../results/ner_entities_with_sentiment.csv"
df.to_csv(output_path, index=False)
output_path

'../results/ner_entities_with_sentiment.csv'

In [9]:
(
    df.groupby(["entity", "label"]).size()
      .reset_index(name="count")
      .sort_values("count", ascending=False)
)

,entity,label,count
0,5G,MISC,1
1,Bouygues Telecom,ORG,1
2,Free,ORG,1
3,Lyon,LOC,1
4,Marseille,LOC,1
5,Orange,ORG,1
6,Paris,LOC,1
7,SFR,ORG,1
